# 題目 - Find the constructions !
data：2022年PTT美食版共四個月的資料


> 請在資料中找出下列5個two-slot-construction（2個變項)：

1.   X + 的 + Y + 的
2.   不 + X1 + 不 + X2
3.   X + 了 + Y + 了
4.   一 + q1 + 一 + q2
5.   好 + v1 + 好 + v2

需要特別留意的是，我們並非在尋找四字格，X, Y等這些變項都是word (not character)！



---





資料來源參考晴方學姊:
[北大構式語料庫](http://ccl.pku.edu.cn/ccgd/)共收錄了113個簡體中文構式。
晴方學姊以PTT 2020年整年的15個版去篩選（台灣繁體語用），其中共有25個構式有超過100筆，最後選出12個有多義的構式進行研究。
[Demo App ](https://andreashih.shinyapps.io/cca-python1/?fbclid=IwAR1Ii3jl4rmfd4bEQNE3TqmIUw55ctqiO1mRdYLht-54MKgeVgHdgUvJhzo)









##（1) 前處理（10%)

資料格式為json檔，每一篇PTT貼文會有title跟content，其中content包含po文內容跟留言，都已經用換行符號斷句。

請用上週的方式清理資料，並用ckip進行斷詞。

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import json
import re
import pandas as pd
from collections import Counter

In [ ]:
with open('/content/gdrive/MyDrive/NTU GIL/CLLT ta/cllt2023/ppt_food_2022.json', 'r') as f:
    data = json.load(f) 

In [ ]:
! pip3 install -U ckip-transformers
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger

In [ ]:
data[0]

In [ ]:
# 要清理掉的regex
trash_regex = "@\S+|https?:\S+|http?:\S|[^\u4E00-\u9FD5]"

# title, content都取出並清理乾淨
txt = []
for data_x in data:
  data_x['title'] = re.sub(trash_regex, '', data_x['title'])
  txt.append(data_x['title'])

  for con in data_x['content']:
    con = re.sub(trash_regex, '', con)
    txt.append(con)
len(txt)

114531

In [ ]:
txt[:10]

['食記台南柳營露水雞鄉野小路間的美食',
 '餐廳名稱台南柳營露水雞',
 '消費時間年月',
 '地址台南市柳營區旭山里山子腳鄰之號縣道號處',
 '電話',
 '營業時間週一公休',
 '每人平均價位元左右',
 '可否刷卡否',
 '有無包廂無',
 '推薦菜色枸杞雞茄餅']

In [ ]:
# tokenize function
## ws: [['A','B'],['a','b','c']]  
## output:['A B', 'a b c']

def tokenize(texts, ws_driver, batch_size=512, max_length=256):
    output=[]
    ws  = ws_driver(texts, use_delim=False, 
                batch_size = batch_size,
                 max_length = max_length
                )
    output = [" ".join(ls) for ls in ws]
    return ws, output

In [ ]:
'''執行階段記得選GPU 不然會很久很久很久'''
ws_driver = CkipWordSegmenter(device=0)
ws, output = tokenize(txt, ws_driver)
output[:10]

Inference: 100%|██████████| 190/190 [14:21<00:00,  4.53s/it]


['食記 台南 柳營 露水雞 鄉野 小路 間 的 美食',
 '餐廳 名稱 台南 柳營 露水雞',
 '消費 時間 年 月',
 '地址 台南市 柳營區 旭山里 山子腳鄰 之號 縣道號 處',
 '電話',
 '營業 時間 週一 公休',
 '每 人 平均 價位 元 左右',
 '可否 刷卡 否',
 '有無 包廂 無',
 '推薦 菜色 枸杞 雞 茄餅']

In [ ]:
import pickle

with open('./tokenized.pickle', 'wb') as f:
    pickle.dump(output, f)

## (2) 找出構式！(40%)


整理出一個構式DataFrame，包含7個欄位：

*   form: construction pattern
*   cnstr: 資料中match到的構式（10%)
*   cntxt: context，出現構式的那整個句子 (10%)
*   var_1: 第一個變項 (5%)
*   var_2: 第二個變項 (5%)
*   var1_pos: 第一個變項的詞性 (5%)
*   var2_pos: 第二個變項的詞性 (5%)

請注意！
詞性的篩選請至少以「資料中match到的構式」為單位，例如「吃 的 喝 的」要整組進行pos-tagging，再取出變項「吃」跟「喝」的詞性，不可以只有單獨tag「吃」跟「喝」。

> ⚠️⚠️⚠️``print df[300:310]``⚠️⚠️⚠️

In [ ]:
# Match cnstr patterns in corpus
def get_matched_segment(form, pat, content):

  matcheds = {}
  for j, k in zip(form, pat):
    pattern = re.compile(k)
    if k not in matcheds: matcheds[j] = []
    
    # Get all cnstr from all sentences
    candidates, context = [], []
    for sent in content:
      for c in pattern.finditer(sent): 
        candidates.append(c[0])
        context.append(sent)

    for x in range(len(candidates)): 
      matcheds[j].append((candidates[x],context[x]))

  return matcheds

# Make cnstr_pos df
def get_cnstr_df(matched, pos_driver):
  keys = list(matched.keys())
  values = list(matched.values())

  # get variable indexes
  var_idx = []
  for k in keys:
      split_k = k.split('+') 
      var = []
      for idx, word in enumerate(split_k):
        if (re.match(r'[a-zA-Z]', word)):
          var.append(idx)
      var_idx.append(var)

  # add cnstr pos
  dfs = []
  for i in range(len(keys)):   
      for (mat, cntxt) in values[i]:
        split_m = mat.split(' ')
        split_m = [m for m in split_m if m != '']
        cnstr_pos = pos_driver([mat], use_delim=False, show_progress=False)
        cnstr_pos = cnstr_pos[0]
        cnstr_pos = [m for m in cnstr_pos if m != 'WHITESPACE']

        dfs.append({
              'form': keys[i], 
              'cnstr': mat,
              'cntxt': cntxt,
              'var1': split_m[var_idx[i][0]],
              'var2': split_m[var_idx[i][1]],
              'var1_pos': cnstr_pos[var_idx[i][0]],
              'var2_pos': cnstr_pos[var_idx[i][1]]
              })  

  cnstr_df = pd.DataFrame(dfs)
  
  return cnstr_df

In [ ]:
cnstrs = [
    ["X", "+", "的", "+", "Y", "+", "的"],
    ["不", "+", "X1", "+", "不", "+", "X2"],
    ["g1", "+", "了", "+", "g2", "+", "了"],
    ["好", "+", "v1", "+", "好", "+", "v2"],
    ["一", "+", "q1", "+", "一", "+", "q2"]
]

forms = [''.join(x) for x in cnstrs]
forms

['X+的+Y+的', '不+X1+不+X2', 'g1+了+g2+了', '好+v1+好+v2', '一+q1+一+q2']

In [ ]:
for cn in cnstrs:
  for idx, item in enumerate(cn):
    if re.match(r'[a-zA-Z]', item):
      cn[idx] = "\w+"

for cnstr in cnstrs:
  for cn in cnstr:
    if cn == '+':
      cnstr.remove(cn)

forms_pat = ['\s'.join(x) for x in cnstrs]
forms_pat = ['(\s'+ x +'\s)' for x in forms_pat]
#forms_pat = [re.sub('\+', '', x) for x in forms_pat]
forms_pat


['(\\s\\w+\\s的\\s\\w+\\s的\\s)',
 '(\\s不\\s\\w+\\s不\\s\\w+\\s)',
 '(\\s\\w+\\s了\\s\\w+\\s了\\s)',
 '(\\s好\\s\\w+\\s好\\s\\w+\\s)',
 '(\\s一\\s\\w+\\s一\\s\\w+\\s)']

In [ ]:
matched = get_matched_segment(forms, forms_pat, output)
matched

In [ ]:
for k, v in matched.items():
  print(f'{k}: {len(v)}')

X+的+Y+的: 258
不+X1+不+X2: 47
g1+了+g2+了: 17
好+v1+好+v2: 27
一+q1+一+q2: 145


In [ ]:
pos_driver = CkipPosTagger(device=0)

cnstr_df = get_cnstr_df(matched, pos_driver)
cnstr_df.head()

,form,cnstr,cntxt,var1,var2,var1_pos,var2_pos
0,X+的+Y+的,下 的 阿將 的,位在 阿里山 山腳 下 的 阿將 的 家 咖啡館 是 一 位 鄒族 藝術家 阿將 花 了 多...,下,阿將,Ng,Nc
1,X+的+Y+的,喜歡 的 台南 的,不過 粉腸 我 倒是 挺 喜歡 的 台南 的 粉腸 並 不 是 豬小腸 而是 有點 類似 香...,喜歡,台南,VK,DE
2,X+的+Y+的,嫩 的 意外 的,香 也 頗 嫩 的 意外 的 是 還 有 不少 蛤蜊 雖 個頭 不 大 但是 頗 新鮮,嫩,意外,VH,VK
3,X+的+Y+的,大 的 蝦醬 的,蝦醬 空心菜 也 是 相當 經典 的 泰國 菜 份量 滿 大 的 蝦醬 的 味道 很 不錯 ...,大,蝦醬,VH,Na
4,X+的+Y+的,奶油 的 熱熱 的,中間 夾有 冰 奶油 的 熱熱 的 菠蘿油 可以 的話 真的 是 要 熱騰騰 的 吃 才 是...,奶油,熱熱,Na,DE


In [ ]:
cnstr_df.shape

(494, 7)

In [ ]:
cnstr_df[300:310]

,form,cnstr,cntxt,var1,var2,var1_pos,var2_pos
300,不+X1+不+X2,不 油 不 膩,但是 味道 適中 不 油 不 膩 算是 可以 吃 很多 碗 的 類型,油,膩,Na,VH
301,不+X1+不+X2,不 油 不 膩,食記 台東 池上 好 煎炸 創意 巨大 炸 春捲 不 油 不 膩 更 顯 清爽,油,膩,Na,VH
302,不+X1+不+X2,不 吃 不 知道,奶酥堡 他 是 寫 熟客 必 點 拉 不 吃 不 知道 一 吃 嚇一跳 超棒,吃,知道,VC,VK
303,不+X1+不+X2,不 吃 不 知道,高麗菜 蛋餅 又 一 個 不 吃 不 知道 吃到 嚇一跳 高麗菜 好 鮮甜 吃 了 幾 次 ...,吃,知道,VC,VK
304,不+X1+不+X2,不 鹹 不 油,為 沒有 什麼 鹹味 可能 適合 歲 的 老 杯杯 吃 不 鹹 不 油 一定 很 養生,鹹,油,VH,Na
305,g1+了+g2+了,棚 了 到 了,這 就 滿 普通 的 然後 洋芋 貌似 跑 錯 棚 了 到 了 羊排 這,棚,到,Na,Caa
306,g1+了+g2+了,甜味 了 咬 了,這時 只 差 甜味 了 咬 了 一 口 鰻魚 後 炙燒 後 的 表皮 因為 黑糖 而 更 酥...,甜味,咬,Na,Di
307,g1+了+g2+了,吃 了 不少 了,下車 的 地方 是 新潟 的 巴士 中心 也 就 是 巴士 總站 本 以為 早餐 已經 吃 ...,吃,不少,VC,Neqa
308,g1+了+g2+了,好吃 了 加 了,椰菜醬 吃起來 會 有 一點點 淡淡 咖哩 的 錯覺 不用 加 醬 就 很 好吃 了 加 了...,好吃,加,VH,Di
309,g1+了+g2+了,精瘦 了 放 了,嘉義 黃牛 菲力 威靈頓 台灣 牛 的 菲力 肉質 又 更 精瘦 了 放 了 少許 肥 肝 ...,精瘦,放,VH,Di


#### 先看一下篩選前的結果（作業沒有這部分）

In [ ]:
form_c = list(set(cnstr_df.form.to_list()))

stats_c = cnstr_df.groupby('form')['cnstr'].value_counts()
for k in form_c:
  print(f'{k}: {stats_c[k].sum()}')

g1+了+g2+了: 17
好+v1+好+v2: 27
一+q1+一+q2: 145
不+X1+不+X2: 47
X+的+Y+的: 258


In [ ]:
stats_c['一+q1+一+q2']

cnstr
 一 顆 一 顆     8
 一 個 一 個     7
 一 期 一 會     7
 一 層 一 層     6
 一 口 一 口     4
            ..
 一 鍋 一 燒     1
 一 間 一 間     1
 一 陣 一 陣     1
 一 隻 一 歲     1
 一 面 一 面     1
Name: cnstr, Length: 88, dtype: int64

In [ ]:
stats_c['不+X1+不+X2']

cnstr
 不 油 不 膩       9
 不 甜 不 膩       3
 不 吃 不 知道      2
 不 肥 不 膩       2
 不 苦 不 澀       2
 不 一樣 不 太      1
 不 一樣 不 知道     1
 不 不 不 前面      1
 不 到 不 是       1
 不 多 不 代表      1
 不 多 不 清楚      1
 不 大 不 到       1
 不 太 不 習慣      1
 不 好 不 壞       1
 不 好吃 不 要      1
 不 快 不 慢       1
 不 是 不 好吃      1
 不 是 不 知       1
 不 熟 不 一樣      1
 不 燥熱 不 是      1
 不 肥 不 瘦       1
 不 腥 不 臊       1
 不 規則 不 對稱     1
 不 賺 不 賠       1
 不 辣 不 吃       1
 不 辣 不 太       1
 不 辣 不 甜       1
 不 辣 不 酸       1
 不 重 不 喜歡      1
 不 難吃 不 吃      1
 不 馬虎 不 是      1
 不 高 不 敢       1
 不 鹹 不 油       1
 不 黏 不 膩       1
Name: cnstr, dtype: int64

## (3) 篩選詞性，計算frequency（20%)

> 詞性的篩選要根據代號：
1.   cnstr_1 = X + 的 + Y + 的
2.   cnstr_2 = 不 + X1 + 不 + X2
3.   cnstr_3 = X + 了 + Y + 了
4.   cnstr_4 = 一 + q1 + 一 + q2
5.   cnstr_5 = 好 + v1 + 好 + v2



*    X, Y: 不限詞性且兩個常項可以是不同詞性
*    X1, X2: 不限詞性但兩個常項必須為同一詞性
*    v1, v2: 兩個常項都必須為動詞（V[A-Z]）
*    q1, q2: 兩個常項都必須為量詞（Nf)

透過詞性將不符合的row排除後，計算出 **每個構式總計出現的次數 (10%)** 以及 **出現次數最多的前10名 (10%)** 。

In [ ]:
cnstr_pos_df = cnstr_df.copy()
for idx, row in cnstr_pos_df.iterrows():
    # v1, v2
    if re.findall('v\d', row['form']):
      if not (re.findall('V[A-Z]+', row['var1_pos']) and re.findall('V[A-Z]+', row['var2_pos'])):
        cnstr_pos_df.drop(idx, inplace=True)

    # q1, q2 (try Nf or N[a-z])
    elif re.findall('q\d', row['form']):
      if not ((row['var1_pos'] == 'Nf') and (row['var2_pos'] == 'Nf')):
        cnstr_pos_df.drop(idx, inplace=True)
    
    # X1, X2
    elif re.findall('X\d', row['form']):
      if row['var1_pos'][0] != row['var2_pos'][0]:
        cnstr_pos_df.drop(idx, inplace=True)

cnstr_pos_df.head()

,form,cnstr,cntxt,var1,var2,var1_pos,var2_pos
0,X+的+Y+的,下 的 阿將 的,位在 阿里山 山腳 下 的 阿將 的 家 咖啡館 是 一 位 鄒族 藝術家 阿將 花 了 多...,下,阿將,Ng,Nc
1,X+的+Y+的,喜歡 的 台南 的,不過 粉腸 我 倒是 挺 喜歡 的 台南 的 粉腸 並 不 是 豬小腸 而是 有點 類似 香...,喜歡,台南,VK,DE
2,X+的+Y+的,嫩 的 意外 的,香 也 頗 嫩 的 意外 的 是 還 有 不少 蛤蜊 雖 個頭 不 大 但是 頗 新鮮,嫩,意外,VH,VK
3,X+的+Y+的,大 的 蝦醬 的,蝦醬 空心菜 也 是 相當 經典 的 泰國 菜 份量 滿 大 的 蝦醬 的 味道 很 不錯 ...,大,蝦醬,VH,Na
4,X+的+Y+的,奶油 的 熱熱 的,中間 夾有 冰 奶油 的 熱熱 的 菠蘿油 可以 的話 真的 是 要 熱騰騰 的 吃 才 是...,奶油,熱熱,Na,DE


In [ ]:
cnstr_pos_df.shape

(333, 7)

In [ ]:
form_k = list(set(cnstr_pos_df.form.to_list()))

stats = cnstr_pos_df.groupby('form')['cnstr'].value_counts()
for k in form_k:
  print(f'{k}: {stats[k].sum()}')

g1+了+g2+了: 17
好+v1+好+v2: 16
一+q1+一+q2: 20
不+X1+不+X2: 22
X+的+Y+的: 258


In [ ]:
stats['X+的+Y+的'][:10]

cnstr
 外婆 的 茶屋 的      4
 烤 的 非常 的       2
 一樣 的 一樣 的      1
 一樣 的 口感 的      1
 一樣 的 名稱 的      1
 一樣 的 外婆 的      1
 一般 的 民宅 的      1
 一般 的 金鑽 的      1
 一點 的 炒飯組 的     1
 上 的 選擇 的       1
Name: cnstr, dtype: int64

In [ ]:
stats['一+q1+一+q2'][:10]

cnstr
 一 顆 一 顆     8
 一 個 一 個     7
 一 塊 一 塊     4
 一 隻 一 歲     1
Name: cnstr, dtype: int64

In [ ]:
stats['g1+了+g2+了'][:10]

cnstr
 久 了 約 了      1
 吃 了 不少 了     1
 吃完 了 忘 了     1
 好吃 了 加 了     1
 忘 了 拍 了      1
 拿光 了 問 了     1
 棚 了 到 了      1
 甜味 了 咬 了     1
 用 了 加 了      1
 的 了 吃 了      1
Name: cnstr, dtype: int64

In [ ]:
stats['好+v1+好+v2'][:10]

cnstr
 好 久 好 久      1
 好 乖 好 慵懶     1
 好 可惜 好 想     1
 好 好吃 好 香     1
 好 嫩 好 新鮮     1
 好 彈 好 甜      1
 好 想 好 想      1
 好 滑 好 順口     1
 好 滿足 好 飽     1
 好 濃 好 香      1
Name: cnstr, dtype: int64

In [ ]:
stats['不+X1+不+X2'][:10]

cnstr
 不 甜 不 膩      3
 不 吃 不 知道     2
 不 肥 不 膩      2
 不 苦 不 澀      2
 不 多 不 清楚     1
 不 好 不 壞      1
 不 快 不 慢      1
 不 熟 不 一樣     1
 不 肥 不 瘦      1
 不 腥 不 臊      1
Name: cnstr, dtype: int64

## (4) 分析構式及語境（30%)
> 挑選3個構式，對其進行分析（可以是質性或量化分析）。(各10%)

例如：
分析 A 構式
- A 構式常出現在什麼discourse context （提醒、建議、論述、警告、批判等等），可能是什麼原因？
- 在一個句子中，常跟A構式共同出現的字詞、詞性、人稱、動詞類型、其他構式等等，可能是什麼原因？
- 用CwnGraph或dependency parser，分析 A 構式內部（那4個token）的構詞關係、語意關係、句法結構... 

`不一定要針對上述項目做分析，請多觀察你整理的資料，並且佐以查到的語言學、構式相關資料，發揮想像力＆平時做研究的分析能力！`


[Deborah's answer](https://github.com/lope-classroom/hw7_construction-deborahwatty/blob/main/HW7_construction_deborahwatty.ipynb)

[Allen's answer](https://github.com/lope-classroom/hw7_construction-allenwch/blob/main/HW7_construction_%E7%86%8A%E5%81%89%E5%82%91.ipynb)